In [18]:
-- Select some customer data at random to make sure your integration is set up correctly
SELECT * 
FROM public.customer
LIMIT 10;

,customer_id,first_name,last_name,email
0,1,Ursola,Purdy,upurdy0@cdbaby.com
1,2,Ruthanne,Vatini,rvatini1@fema.gov
2,3,Reidar,Turbitt,rturbitt2@geocities.jp
3,4,Rich,Kirsz,rkirsz3@jalbum.net
4,5,Carline,Kupis,ckupis4@tamu.edu
5,6,Kandy,Adamec,kadamec5@weather.com
6,7,Jermain,Giraudeau,jgiraudeau6@elpais.com
7,8,Nolly,Bonicelli,nbonicelli7@examiner.com
8,9,Phebe,Curdell,pcurdell8@usa.gov
9,10,Euell,Guilder,eguilder9@themeforest.net


## Task 2. What do the ids in the status tables mean?

In [4]:
-- Look up the meaning of address_status ids
--SELECT * 
-- FROM address_status

-- Look up the meaning of order_status ids
SELECT * 
FROM order_status

,status_id,status_value
0,1,Order Received
1,2,Pending Delivery
2,3,Delivery In Progress
3,4,Delivered
4,5,Cancelled
5,6,Returned


## Task 3. How does the order_history table work?

In [5]:
-- Select an order_id from order_history with a status_id corresponding to `Returned`
-- SELECT *
-- FROM order_history 
-- WHERE status_id = 6
-- LIMIT 1;

-- Select all data in order_history with the order_id you found with the query above.
SELECT *
FROM order_history
WHERE order_id = 4412;

,history_id,order_id,status_id,status_date
0,4411,4412,1,2021-01-13 10:54:14.267000+00:00
1,10842,4412,2,2021-01-14 07:27:36.267000+00:00
2,14396,4412,3,2021-01-14 07:49:58.267000+00:00
3,19564,4412,4,2021-01-16 04:00:18.267000+00:00
4,22149,4412,6,2021-01-13 10:56:33.267000+00:00


## Task 4. How many errors are returned by users?

In [6]:
SELECT COUNT(*)
FROM order_history
WHERE status_id = 6;

,count
0,200


## Task 5. How to join the order_history and cust_order tables?

In [17]:
SELECT *
FROM cust_order AS c
JOIN order_history AS o
USING (order_id)
WHERE c.order_id = 4412;

,order_id,order_date,customer_id,shipping_method_id,dest_address_id,history_id,status_id,status_date
0,4412,2021-01-13 01:37:21.267000+00:00,96,1,701,4411,1,2021-01-13 10:54:14.267000+00:00
1,4412,2021-01-13 01:37:21.267000+00:00,96,1,701,10842,2,2021-01-14 07:27:36.267000+00:00
2,4412,2021-01-13 01:37:21.267000+00:00,96,1,701,14396,3,2021-01-14 07:49:58.267000+00:00
3,4412,2021-01-13 01:37:21.267000+00:00,96,1,701,19564,4,2021-01-16 04:00:18.267000+00:00
4,4412,2021-01-13 01:37:21.267000+00:00,96,1,701,22149,6,2021-01-13 10:56:33.267000+00:00


## Task 6. Do a significant number of users return multiple orders?

In [16]:
SELECT c.customer_id,
		COUNT(o.status_id) AS returned_orders
FROM cust_order AS c
JOIN order_history AS o
USING (order_id)
WHERE o.status_id = 6
GROUP BY c.customer_id
ORDER BY returned_orders DESC;

,customer_id,returned_orders
0,60,3
1,107,3
2,83,2
3,42,2
4,94,2
...,...,...
171,92,1
172,1598,1
173,679,1
174,337,1


## Task 7. Who are the customers returning more orders than usual?

After showing that there are some users returning more orders than others, we would like to create a list of users to investigate or contact and get to the bottom of why these users return more orders than others.

In [15]:
SELECT customer.first_name,
		customer.last_name,
		customer.email,
		c.customer_id,
		COUNT(o.order_id) AS returned_orders
FROM cust_order AS c
JOIN order_history AS o
USING (order_id)
LEFT JOIN customer
USING (customer_id)
WHERE o.status_id = 6
GROUP BY customer.first_name, customer.last_name, customer.email, c.customer_id
HAVING COUNT(o.order_id) >= 2
ORDER BY returned_orders DESC;

,first_name,last_name,email,customer_id,returned_orders
0,Kincaid,De Avenell,kdeavenell1n@wikia.com,60,3
1,Estelle,Alton,ealton2y@vimeo.com,107,3
2,Ruthanne,Vatini,rvatini1@fema.gov,2,2
3,Leanor,Ortler,lortler2l@wikimedia.org,94,2
4,Garold,Kobisch,gkobisch1g@cdc.gov,53,2
5,Levy,Thacker,lthacker1f@hc360.com,52,2
6,Anna-diana,Tilburn,atilburn15@simplemachines.org,42,2
7,Andrei,Jefferson,ajeffersonh@live.com,18,2
8,Debbi,Huyghe,dhuyghef@dot.gov,16,2
9,Mariette,Tulley,mtulley9j@yahoo.co.jp,344,2
